<a href="https://colab.research.google.com/github/leotuni/Nicolas-stuff/blob/master/SCTC_leo_no_note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title imports
import datetime as dt
import pandas as pd
import pandas_datareader.data as web
import numpy as np 
from itertools import product 


#display tools 
from IPython.display import display
pd.options.display.max_columns = None
# style.use('ggplot') #this is a cool style

#Plots
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [ ]:
#@title Dataframe setup  { form-width: "30px" }
#the date in a nice date format
start_date_data = '2018-03-14' #@param {type:"date"} 
start = dt.datetime.strptime(start_date_data, '%Y-%m-%d') 
stop = dt.datetime.now()

#the symbols you gave me 
symbols = ['BTC-EUR','ETH-EUR', 'LTC-EUR' , 'XRP-EUR', 'BCH-EUR','MIOTA-EUR' ]


#create a dictionary with all the stocks Q: do you prefer to work with lists of dictionaries ? 
dict_of_stock = { s: web.DataReader(s, "yahoo", start, stop) for s in (symbols) }


In [ ]:
#@title Functions SMA part (set BUY as + and Sell as - In dataframe)  { form-width: "30px" }

def conditions_fun (df, col1, col2, choices= '''[df['Close'], -df['Close']]''', defa = 0): 
    ''' 
      dataframe, col2, col2, choice after condition, defa = 0
      returns pandas series, index taken from df. 
      Choices decide if we assing a postive or a negative number when buy or sell condition is met
    
    '''
    conditions = [
                    ((df[col1] > df[col2]) & (df[col1].shift(1) < df[col2].shift(1))), #buy condition
                    ((df[col1] < df[col2]) & (df[col1].shift(1) > df[col2].shift(1))), #sell condition
                    ]
    return  (pd.Series(np.select(conditions, choices, default=defa), index=df.index)) #default value we do nothing



In [ ]:
#@title SMA main: different rolling window sizes { form-width: "30px" }

#Only getting close since it is all we are using 
Close=dict_of_stock['BTC-EUR']['Close']
Close=Close.groupby(Close.index).first() #removes the duplicate indexes

# made them from 2-6 , 7-11 just cause it is easier to see. made into strings cause im using them as col_names
SMA1 = list(map(str, range(2, 30))) #
SMA2 = list(map(str, range(31, 60)))

# # table = [ Close.rolling(s,min_periods=1).mean() for s in (np.arange(1,10))]
data = pd.DataFrame.from_dict({ s: Close.rolling(int(s),min_periods=1).mean() for s in SMA1 + SMA2 }) 
data.insert(0, 'Close', Close)


#This creates a df for the buy and sell values based on the conditions
buy_sell_df = pd.DataFrame.from_dict({  sma1 + '_' + sma2: conditions_fun (data, sma1, sma2 , choices= [data['Close'], -data['Close']])  for sma1, sma2  in product(SMA1, SMA2) })
buy_sell_df.insert(0, 'Close', Close)

# other solution, currently not using it. 
# buy_sell_df.where(buy_sell_df.gt(0).cummax(), 0, inplace=True) #https://stackoverflow.com/questions/62533032/pandas-replace-col-values-before-first-postive-with-0/62533267#62533267
 

#if last non-zero element is negative(buying), replace with 0
s = buy_sell_df.iloc[::-1] #inverts the dataframe 
buy_sell_df[s.lt(0).cumsum().eq(1) & (~s.gt(0).cummax())] = 0

#if first non-zero elemnt is positive (selling), replace with 0
s= buy_sell_df.iloc[::] #normal dataframe 
buy_sell_df[s.gt(0).cumsum().eq(1) & (~s.lt(0).cummax())] = 0


# #look at these two 
# data
# pd.set_option('display.max_rows', None)

# buy_sell_df[:]



In [ ]:
#@title PLOT: Frequency of buy sell events for SMA pairs

mask_gt=buy_sell_df.gt(0)
mask_lt=buy_sell_df.lt(0)
Data=buy_sell_df.drop(['Close'], axis=1)
Data[mask_gt]=1
Data[mask_lt]=-1

fig = go.Figure(data=go.Heatmap(
        z=Data.to_numpy().transpose(),
        x=Data.index,
        y=Data.columns,
        colorscale='Viridis'))

fig.update_layout(
    title='Buy (B) sell(Y) shown over time',
    xaxis_nticks=36)

fig.show()

In [ ]:
#@title PLOT:  Heatmap #of times a buy/sell is triggered 
#since they area all odd this makes no sense. 

# Counts= buy_sell_df.astype(bool).sum(axis=0).drop('Close', inplace=True)
Counts = pd.DataFrame( { 'Position': buy_sell_df.astype(bool).sum(axis=0)[1:].index, 'Value': buy_sell_df.astype(bool).sum(axis=0)[1:]  } ) 
Counts.index=pd.MultiIndex.from_tuples(Counts.Position.str.split('_').apply(tuple))

c=Counts.Value.unstack(0)
#make the colums into integers
c.columns = pd.to_numeric(c.columns, errors = 'coerce')
c.sort_index(axis=1, inplace= True)

#make the index ordered 
c.index= pd.to_numeric(c.index, errors = 'coerce')
c.sort_index(axis=0, inplace=True)


ax = sns.heatmap(c)
plt.ylabel('SMA1')
plt.xlabel('SMA2')
plt.title('Heatmap #of conditions met')



The best SMA: evaluations are - : 



In [ ]:
#@title PLOT Heatmap of profit for SMA: https://stackoverflow.com/questions/62640043/python-create-heatmap-using-string-in-col1-x-y-as-coordinates
Profit = pd.DataFrame( { 'Position': buy_sell_df.sum()[1:].index, 'Value': buy_sell_df.sum()[1:]  } ) 


Profit.index=pd.MultiIndex.from_tuples(Profit.Position.str.split('_').apply(tuple))
s=Profit.Value.unstack(0)

s.columns = pd.to_numeric(s.columns, errors = 'coerce')
s.sort_index(axis=1, inplace= True)
s.index= pd.to_numeric(s.index, errors = 'coerce')
s.sort_index(axis=0, inplace=True)


ax = sns.heatmap(s)
plt.ylabel('SMA1')
plt.xlabel('SMA2')
plt.title('Heatmap Profit (-buy + sell)')

